### Twitter Dev
1. Connect to redis data store
2. Build dataframe from reading redids datastore
3. Get ticker list
4. create attribute filter, setup stream and log
5. Setup log schedule
6. Run stream

In [1]:
#Imports
import redis
import urllib.request
import json
import numpy as np
import pandas as pd
import schedule
import time
import boto3
import datetime

#Twitter requirements
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
from tweepy import API

#Log Requirements
import mysql.connector
from sqlalchemy import create_engine
from sqlalchemy import inspect
from sqlalchemy import MetaData
from sqlalchemy import Table

kinesis = boto3.client('kinesis', region_name='us-east-1')


#Connect to Redis-DataStore
REDIS = redis.Redis(host='data_store')

In [2]:
#Get Environment Variables
ACCESS_TOKEN = %env ACCESS_TOKEN
ACCESS_TOKEN_SECRET = %env ACCESS_TOKEN_SECRET
CONSUMER_KEY = %env CONSUMER_KEY
CONSUMER_SECRET = %env CONSUMER_SECRET

In [3]:
User = %env DB_USER
password = %env DB_PWD
dbname = %env DB_NAME

#### Build Company DataFrame

In [4]:
companies = json.loads(REDIS.get('companies').decode())
company_df = pd.DataFrame.from_dict(companies, orient='index')
company_df.index.name = 'Ticker'
company_df.columns=['Company']
#Add code to add ticker symbol
company_df['tweet_ticker']=company_df.index.map(lambda x: '$'+x)

company_df.head()

,Company,tweet_ticker
Ticker,,
AAPL,Apple,$AAPL
FB,Facebook,$FB
GOOG,Google,$GOOG


In [5]:
tickers = company_df['tweet_ticker'].tolist()

In [6]:
#Want to create a simple list, but how do I handle the users sub?
attributes = ['created_at',
             'id_str',
             'text',
              'quote_count',
              'reply_count',
              'retweet_count',
              'favorite_count',
              'retweeted',
              'lang',
              ['user','name'],
              ['user','followers_count'],
              ['user','statuses_count'],
              ['user','screen_name']
               ]
def filter_attr(data):
    output = {}
    #Choose filter attributes
    for element in attributes:
        if isinstance(element, str):
            output[element]=data[element]
        #Handle Nested Attributes
        else:
            string = str(element[0])+'_'+str(element[1])
            output[string]=data[element[0]][element[1]]
  
    #Need to also add the company name to output dictionary.
    #Add all companies tweet applies to in list
    attached_company = []
    
    for company in tickers:
        if data['text'].find(company) > -1:
            attached_company.append(company[1:])
            
    
    output['Company']=attached_company             
    
    return output

#This is a basic listener that just prints received tweets to stdout.
class TweetListener(StreamListener):
    
    def on_data(self, data):
        try:
            datajson = json.loads(data)
            filtered = filter_attr(datajson)
            #Check to see if a valid tweet
            if filtered['Company'] and filtered['lang']=='en':
                
                print(filtered)
                #Add counter to count stocks. 
                REDIS.incr('Twitter_Stock_Count')
                #---------- Insert to Kinesis Stream --------------
                #response = kinesis.put_record(StreamName="Twitter_Stream", Data=json.dumps(filtered), PartitionKey="partitionkey")
                return True
            
        except Exception as e:
            print(e)

    def on_error(self, status):
        print('The error code is: ' + repr(status))
        #Continue even if there is an error
        #Need to publish the error to the redies error handler in manager
        #Need to think about if I want to continue running
        return True


#Setup Log

global past_tweet_count
past_tweet_count = 0

#Setup Database
engine = create_engine('mysql+mysqlconnector://{}:{}@db:3306/{}'.format(User, password, dbname), echo=False)
conn = engine.connect()

#Create table object
meta = MetaData(engine,reflect=True)
table = meta.tables['log']

def log():
    #Need to log: Time, Source, Current Count, Count Diff
    now = datetime.datetime.now()
    current_tweet_count = int(REDIS.get('Twitter_Stock_Count'))
    
     
    global past_tweet_count
    tweet_count_diff = current_tweet_count - past_tweet_count
    past_tweet_count = current_tweet_count
    
    #Need to log these items to a database.
    ins = table.insert().values(
            log_time = now,
            Source = 'Twitter',
            Current_Count=current_tweet_count,
            Count_Diff=tweet_count_diff
               )
    conn.execute(ins)
    print('Logged Data')

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:80: SADeprecationWarning: reflect=True is deprecate; please use the reflect() method.


In [7]:
REDIS.set('Twitter_Stock_Count', 0)

#This handles Twitter authetication and the connection to Twitter Streaming API
tweetlist = TweetListener(api=API(wait_on_rate_limit=True,wait_on_rate_limit_notify=True))
auth = OAuthHandler(CONSUMER_KEY,CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN,ACCESS_TOKEN_SECRET)
stream = Stream(auth, tweetlist)

#Filters by the ticker names
print('Filtering: ' + str(tickers))
stream.filter(track=tickers, async=True)

#Setup Schedule
schedule.clear()
schedule.every(60).seconds.do(log)

#Execute
while True:
    schedule.run_pending()
    #Cancel Schedule if an error occurs. and stop this loop. 
    time.sleep(1)

Filtering: ['$AAPL', '$FB', '$GOOG']
{'created_at': 'Fri Feb 02 20:57:46 +0000 2018', 'id_str': '959531309566750723', 'text': 'Barclays Raises Facebook $FB Price Target to $225.00 https://t.co/eSifUs8HAu #stocks', 'quote_count': 0, 'reply_count': 0, 'retweet_count': 0, 'favorite_count': 0, 'retweeted': False, 'lang': 'en', 'user_name': 'Chaffey Breeze News', 'user_followers_count': 227, 'user_statuses_count': 326709, 'user_screen_name': 'chaffeybreeze', 'entities_hashtags': [{'text': 'stocks', 'indices': [77, 84]}], 'entities_symbols': [{'text': 'FB', 'indices': [25, 28]}], 'Company': ['FB']}
{'created_at': 'Fri Feb 02 20:57:46 +0000 2018', 'id_str': '959531312251097088', 'text': 'Barclays Raises Facebook $FB Price Target to $225.00 https://t.co/0FDQpkTqAb #markets', 'quote_count': 0, 'reply_count': 0, 'retweet_count': 0, 'favorite_count': 0, 'retweeted': False, 'lang': 'en', 'user_name': 'IRA Market Report', 'user_followers_count': 281, 'user_statuses_count': 279903, 'user_screen_name

{'created_at': 'Fri Feb 02 21:00:18 +0000 2018', 'id_str': '959531949839863808', 'text': 'Cost of Hedging Downside Risk (30 Day)  $SPY $UVXY $TVIX $DJIA $AAPL $XIV $SVXY $QQQ $VXX $VKTX $DIA $SPX $DVAX… https://t.co/1tX1uEbwEl', 'quote_count': 0, 'reply_count': 0, 'retweet_count': 0, 'favorite_count': 0, 'retweeted': False, 'lang': 'en', 'user_name': 'bake mcbride', 'user_followers_count': 7519, 'user_statuses_count': 22644, 'user_screen_name': 'ITRADEVOL', 'entities_hashtags': [], 'entities_symbols': [{'text': 'SPY', 'indices': [40, 44]}, {'text': 'UVXY', 'indices': [45, 50]}, {'text': 'TVIX', 'indices': [51, 56]}, {'text': 'DJIA', 'indices': [57, 62]}, {'text': 'AAPL', 'indices': [63, 68]}, {'text': 'XIV', 'indices': [69, 73]}, {'text': 'SVXY', 'indices': [74, 79]}, {'text': 'QQQ', 'indices': [80, 84]}, {'text': 'VXX', 'indices': [85, 89]}, {'text': 'VKTX', 'indices': [90, 95]}, {'text': 'DIA', 'indices': [96, 100]}, {'text': 'SPX', 'indices': [101, 105]}, {'text': 'DVAX', 'indices':

{'created_at': 'Fri Feb 02 21:03:57 +0000 2018', 'id_str': '959532868904079360', 'text': 'The shorts are running out of analysts to downgrade $AAPL lol', 'quote_count': 0, 'reply_count': 0, 'retweet_count': 0, 'favorite_count': 0, 'retweeted': False, 'lang': 'en', 'user_name': 'Hoosh', 'user_followers_count': 310, 'user_statuses_count': 321, 'user_screen_name': 'RyanHooshmand', 'entities_hashtags': [], 'entities_symbols': [{'text': 'AAPL', 'indices': [52, 57]}], 'Company': ['AAPL']}
{'created_at': 'Fri Feb 02 21:04:05 +0000 2018', 'id_str': '959532901816766464', 'text': 'CityFALCON score gives top rated news for FREE #bigdata $AAPL $TSLA https://t.co/1ALoXRrBJr https://t.co/1f9zAYGkLf', 'quote_count': 0, 'reply_count': 0, 'retweet_count': 0, 'favorite_count': 0, 'retweeted': False, 'lang': 'en', 'user_name': 'CityFALCON Analyst', 'user_followers_count': 840, 'user_statuses_count': 255676, 'user_screen_name': 'BUZZ_Just_In', 'entities_hashtags': [{'text': 'bigdata', 'indices': [47, 55]}

{'created_at': 'Fri Feb 02 21:06:18 +0000 2018', 'id_str': '959533460112232450', 'text': 'Everything he could possibly need at his disposal, yet renders $AAPL worst investment on Wall Street acting as a mo… https://t.co/03WNjJEHec', 'quote_count': 0, 'reply_count': 0, 'retweet_count': 0, 'favorite_count': 0, 'retweeted': False, 'lang': 'en', 'user_name': 'Fahed Taji-Farouki', 'user_followers_count': 191, 'user_statuses_count': 3567, 'user_screen_name': 'ffarouki', 'entities_hashtags': [], 'entities_symbols': [{'text': 'AAPL', 'indices': [63, 68]}], 'Company': ['AAPL']}
Logged Data
{'created_at': 'Fri Feb 02 21:07:02 +0000 2018', 'id_str': '959533642794979328', 'text': 'LIVE: Big tech earnings wrap-up - $#AMZN, $#AAPL, $#GOOGL, $FB https://t.co/2RlZ8eb8df https://t.co/3eTDaIsLli', 'quote_count': 0, 'reply_count': 0, 'retweet_count': 0, 'favorite_count': 0, 'retweeted': False, 'lang': 'en', 'user_name': 'Ryan', 'user_followers_count': 4682, 'user_statuses_count': 3356, 'user_screen_name'

{'created_at': 'Fri Feb 02 21:09:42 +0000 2018', 'id_str': '959534314454216705', 'text': 'RT @philstockworld: Going over our @MoneyTalkGO Portfolio live w/ @kimparlee  @TD_Canada trading $AAPL $IMAX $LB $WPM $SQQQ $GE $ABX #Smart…', 'quote_count': 0, 'reply_count': 0, 'retweet_count': 0, 'favorite_count': 0, 'retweeted': False, 'lang': 'en', 'user_name': 'Kathleen Nkrumah', 'user_followers_count': 0, 'user_statuses_count': 43, 'user_screen_name': 'kathleennkruma3', 'entities_hashtags': [], 'entities_symbols': [{'text': 'AAPL', 'indices': [97, 102]}, {'text': 'IMAX', 'indices': [103, 108]}, {'text': 'LB', 'indices': [109, 112]}, {'text': 'WPM', 'indices': [113, 117]}, {'text': 'SQQQ', 'indices': [118, 123]}, {'text': 'GE', 'indices': [124, 127]}, {'text': 'ABX', 'indices': [128, 132]}], 'Company': ['AAPL']}
{'created_at': 'Fri Feb 02 21:09:45 +0000 2018', 'id_str': '959534328739958785', 'text': 'This is one of the most relevant articles about $FB now. Facebook Reports Solid Fourth Quart

...

{'created_at': 'Fri Feb 02 22:04:59 +0000 2018', 'id_str': '959548225928794112', 'text': 'RT @philstockworld: Going over our @MoneyTalkGO Portfolio live w/ @kimparlee  @TD_Canada trading $AAPL $IMAX $LB $WPM $SQQQ $GE $ABX #Smart…', 'quote_count': 0, 'reply_count': 0, 'retweet_count': 0, 'favorite_count': 0, 'retweeted': False, 'lang': 'en', 'user_name': 'Deidre famulary', 'user_followers_count': 0, 'user_statuses_count': 15, 'user_screen_name': 'deidrefamulary8', 'entities_hashtags': [], 'entities_symbols': [{'text': 'AAPL', 'indices': [97, 102]}, {'text': 'IMAX', 'indices': [103, 108]}, {'text': 'LB', 'indices': [109, 112]}, {'text': 'WPM', 'indices': [113, 117]}, {'text': 'SQQQ', 'indices': [118, 123]}, {'text': 'GE', 'indices': [124, 127]}, {'text': 'ABX', 'indices': [128, 132]}], 'Company': ['AAPL']}
{'created_at': 'Fri Feb 02 22:05:39 +0000 2018', 'id_str': '959548393860321281', 'text': 'Tunisian media protest pressure by police to whitewash news $FB #FB #Journalists #Newsmedia #

{'created_at': 'Fri Feb 02 22:11:04 +0000 2018', 'id_str': '959549757055217665', 'text': 'Keep in mind every time Apple has had a major breakout it always pullback and retests $AAPL - It has not yet retested  the last breakout', 'quote_count': 0, 'reply_count': 0, 'retweet_count': 0, 'favorite_count': 0, 'retweeted': False, 'lang': 'en', 'user_name': 'Chart_Guy', 'user_followers_count': 1, 'user_statuses_count': 8161, 'user_screen_name': 'Chart_Guy', 'entities_hashtags': [], 'entities_symbols': [{'text': 'AAPL', 'indices': [86, 91]}], 'Company': ['AAPL']}
{'created_at': 'Fri Feb 02 22:12:19 +0000 2018', 'id_str': '959550070512332806', 'text': 'WhatsApp is now Facebook’s second-biggest property, followed by Messenger and Instagram - Recode https://t.co/ZgSvV80AEI $FB #investing', 'quote_count': 0, 'reply_count': 0, 'retweet_count': 0, 'favorite_count': 0, 'retweeted': False, 'lang': 'en', 'user_name': 'Passed Pawn Advisors', 'user_followers_count': 2200, 'user_statuses_count': 13705, 'u

{'created_at': 'Fri Feb 02 22:17:49 +0000 2018', 'id_str': '959551455886757888', 'text': 'TD BUY $AAPL at 160.50, Supp 160.50 Resis 179.26 R12  HiLo 62% T1Y 190 buy 1.9 PE 17.4 DIV 2.52% #Apple Inc.… https://t.co/B0hu0dv2Yf', 'quote_count': 0, 'reply_count': 0, 'retweet_count': 0, 'favorite_count': 0, 'retweeted': False, 'lang': 'en', 'user_name': 'TradeFlags.io', 'user_followers_count': 67, 'user_statuses_count': 1925, 'user_screen_name': 'TradeFlags', 'entities_hashtags': [{'text': 'Apple', 'indices': [97, 103]}], 'entities_symbols': [{'text': 'AAPL', 'indices': [7, 12]}], 'Company': ['AAPL']}
{'created_at': 'Fri Feb 02 22:18:16 +0000 2018', 'id_str': '959551569271443456', 'text': 'Track curated news and tweets for multiple tickers - Social Media Stocks https://t.co/7Im4LkzEEQ $FB $LNKD $ZNGA', 'quote_count': 0, 'reply_count': 0, 'retweet_count': 0, 'favorite_count': 0, 'retweeted': False, 'lang': 'en', 'user_name': 'FinBuzz', 'user_followers_count': 1171, 'user_statuses_count': 2139

{'created_at': 'Fri Feb 02 22:22:44 +0000 2018', 'id_str': '959552694271672325', 'text': 'RT @philstockworld: Thursday Failure - Fed and Trump Fail to Boost the Markets $SPY $QQQ $BA $AAPL $GOOGL #Futures #Hedging -- https://t.co…', 'quote_count': 0, 'reply_count': 0, 'retweet_count': 0, 'favorite_count': 0, 'retweeted': False, 'lang': 'en', 'user_name': 'Eugenia catanzaro', 'user_followers_count': 1, 'user_statuses_count': 12, 'user_screen_name': 'Eugeniacatanz11', 'entities_hashtags': [{'text': 'Futures', 'indices': [106, 114]}, {'text': 'Hedging', 'indices': [115, 123]}], 'entities_symbols': [{'text': 'SPY', 'indices': [79, 83]}, {'text': 'QQQ', 'indices': [84, 88]}, {'text': 'BA', 'indices': [89, 92]}, {'text': 'AAPL', 'indices': [93, 98]}, {'text': 'GOOGL', 'indices': [99, 105]}], 'Company': ['AAPL', 'GOOG']}
{'created_at': 'Fri Feb 02 22:23:25 +0000 2018', 'id_str': '959552867043627008', 'text': 'RT @PaulGaumond: A real joke today! #Netflix to green! $AAPL to red! #NYSE #technolo

{'created_at': 'Fri Feb 02 22:25:45 +0000 2018', 'id_str': '959553451205132288', 'text': 'RT @philstockworld: Reviewing our Money Talk Portfolio with 6 Trades that Will Make $68,535 (137%) Over 2 Years (if all goes well) $AAPL $A…', 'quote_count': 0, 'reply_count': 0, 'retweet_count': 0, 'favorite_count': 0, 'retweeted': False, 'lang': 'en', 'user_name': 'LaCa', 'user_followers_count': 103, 'user_statuses_count': 6705, 'user_screen_name': 'LaCaProjscet', 'entities_hashtags': [], 'entities_symbols': [{'text': 'AAPL', 'indices': [131, 136]}], 'Company': ['AAPL']}
{'created_at': 'Fri Feb 02 22:25:48 +0000 2018', 'id_str': '959553466686476290', 'text': 'RT @SteadyTrader: Trading Apple Inc. Stock After Earnings $AAPL https://t.co/t2ZeWrmOuI', 'quote_count': 0, 'reply_count': 0, 'retweet_count': 0, 'favorite_count': 0, 'retweeted': False, 'lang': 'en', 'user_name': 'Pensive', 'user_followers_count': 19384, 'user_statuses_count': 24676, 'user_screen_name': 'Oui4Paris', 'entities_hashtags': []